In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

'c:\\Users\\dpere\\Documents\\JTMT\\education'

In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

In [4]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\background_files\betshemesh_muni\חינוך'.format(education_folder_path),'מצבת מוסדות חינוך תשפג משרדי')
BShemesh = remove_spaces_in_columns(BShemesh)

In [5]:
BShemesh = BShemesh.drop(columns=['שם_מנהל/ת', 'כתובת_מנהל', 'נייד', 'שם_מזכיר/ה', 'נייד.1', 'מפקח', 'קב"ט_מוס"ח'])

# רשימת שמות השכבות (א' עד ו')
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב']

# לולאה שמבצעת את הסכימה והמחיקה עבור כל שכבה
for i, grade in enumerate(grades):
    # איתור העמודות הרלוונטיות לכל שכבה (כולל כיתה מקדמת אם קיימת לאותה שכבה)
    grade_columns = [col for col in BShemesh.columns if col.startswith(f'כיתה_{grade}')]
    
    # הוספת העמודה של הכיתה המקדמת של אותה שכבה אם קיימת
    if f'כיתה_מקדמת.{i}' in BShemesh.columns:
        grade_columns.append(f'כיתה_מקדמת.{i}')
    elif 'כיתה_מקדמת' in BShemesh.columns and i == 0:
        grade_columns.append('כיתה_מקדמת')
    
    # סכימת העמודות אם יש כיתות רלוונטיות לשכבה הזו
    if grade_columns:
        BShemesh[f'כיתה_{grade}'] = BShemesh[grade_columns].sum(axis=1, skipna=True)
        
        # מחיקת העמודות שסוכמו
        BShemesh.drop(columns=grade_columns, inplace=True)

# institutions_table = pd.merge(institutions_table, BShemesh, on='סמל_מוסד', how='outer')
# institutions_table = institutions_table.dropna(axis=1, how='all')

BShemesh.head()

,מגזר,שם_המוסד,סמל_מוסד,שכונה,כתובת,טלפון,פקס,מייל,הערות,גן_טרום_חובה,גן_חובה,גני_חינוך_מיוחד,"סה""כ_גנים","סה""כ_יסודי_מ""מ_וממ""ד","סה""כ_יסודי_חרדי","_סה""כ_על_יסודי_מ""מ_וממ""ד","סה""כ_על_יסודי_חרדי",כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב
0,חרדי,אבי העזרי,227439.00,"רמב""ש א",נחל גילה 10,02-9924870,02-9990179,c@kol-y.com,"ת""ת יסודי - בנים - כיתות א' - ח' - פטור",NaN,NaN,NaN,NaN,NaN,252.00,NaN,NaN,21.00,41.00,24.00,45.00,31.00,37.00,32.00,21.00,0.00,0.00,0.00,0.00
1,"ממ""ד",אגוז,628768.00,גבעת שרת,בר אילן 9,NaN,NaN,NaN,גן תקשורת - חינוך מיוחד,NaN,NaN,7.00,7.00,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,"מ""מ",אגם,665257.00,משקפיים,הרב קוק 8,NaN,NaN,NaN,גן שפה - חינוך מיוחד,NaN,NaN,11.00,11.00,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,"ממ""ד",אהבת ישראל - בנות,246033.00,"רמב""ש א",נחל שורק 15,02-9951447,02-9951429,"pazirashi@gmail.com ,aisrael3@gmail.com","בי""ס יסודי - בנות - כיתות א' - ו'",NaN,NaN,NaN,NaN,398.00,NaN,NaN,NaN,64.00,58.00,72.00,71.00,73.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00
4,"ממ""ד",אהבת ישראל - בנים,194472.00,"רמב""ש א",נחל שורק 14,02-9997465,02-9927465,ai.menahel@gmail.com,"בי""ס יסודי - בנות - כיתות א' - ו'",NaN,NaN,NaN,NaN,354.00,NaN,NaN,NaN,35.00,65.00,66.00,82.00,56.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00
